# Latar Belakang
Mengelola dan memahami konten visual dalam jumlah besar secara efisien, terutama untuk meningkatkan aksesibilitas bagi penyandang disabilitas visual dan mendukung pengelolaan arsip digital secara otomatis


# Import libraries

In [1]:
# Import libraries
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor, BertTokenizer
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
import json

# 1. Load Dataset (Flickr8k)

In [ ]:
dataset_path = ""
images_path = os.path.join(dataset_path, "Images")
captions_path = os.path.join(dataset_path, "captions.txt")

# 2. Preprocess Dataset

In [ ]:
# Kelas Flickr8kDataset untuk memproses dataset gambar dan caption
class Flickr8kDataset(Dataset):
    # Inisialisasi kelas dengan parameter utama
    def __init__(self, captions_dict, images_path, feature_extractor, tokenizer, max_len=128):
        """
        Parameters:
        - captions_dict: Dictionary {image_name: [caption1, caption2, ...]}
        - images_path: Path ke direktori gambar
        - feature_extractor: Model pretrained untuk memproses gambar
        - tokenizer: Tokenizer untuk memproses teks caption
        - max_len: Panjang maksimal token caption
        """
        # Konversi dictionary captions_dict menjadi list pasangan (nama_gambar, caption)
        self.captions = list(captions_dict.items())
        # Simpan direktori gambar
        self.images_path = images_path
        # Simpan feature extractor
        self.feature_extractor = feature_extractor
        # Simpan tokenizer
        self.tokenizer = tokenizer
        # Simpan panjang maksimal token
        self.max_len = max_len

    # Fungsi untuk menghitung total jumlah data
    def __len__(self):
        """
        Returns:
        - Jumlah data dalam dataset
        """
        return len(self.captions)

    # Fungsi untuk mengambil data berdasarkan indeks
    def __getitem__(self, idx):
        """
        Parameters:
        - idx: Indeks data yang ingin diambil
        
        Returns:
        - pixel_values: Tensor gambar yang sudah diproses
        - input_ids: Token ID dari teks caption
        - attention_mask: Mask validasi untuk token caption
        """
        # Ambil nama gambar dan daftar caption berdasarkan indeks
        img_name, captions = self.captions[idx]
        # Buat path lengkap ke gambar
        img_path = os.path.join(self.images_path, img_name)

        # Buka gambar dan konversi ke RGB
        image = Image.open(img_path).convert("RGB")
        # Ekstrak fitur gambar menggunakan feature extractor
        pixel_values = self.feature_extractor(images=image, return_tensors="pt").pixel_values.squeeze()

        # Pilih satu caption secara acak untuk mengurangi beban memori
        caption = np.random.choice(captions)
        # Tokenisasi caption dengan tokenizer
        tokenized_caption = self.tokenizer(
            caption,
            padding="max_length",    # Tambahkan padding untuk mencapai panjang max_len
            truncation=True,         # Potong caption jika terlalu panjang
            max_length=self.max_len, # Panjang maksimal caption
            return_tensors="pt",     # Kembalikan tensor PyTorch
        )
        # Ambil token ID dari caption
        input_ids = tokenized_caption.input_ids.squeeze()
        # Ambil attention mask dari caption
        attention_mask = tokenized_caption.attention_mask.squeeze()

        # Kembalikan tensor gambar, token ID, dan attention mask
        return pixel_values, input_ids, attention_mask

In [ ]:
# Load captions dari file teks ke dalam dictionary
captions_dict = {}

# Buka file captions.txt untuk membaca data
with open(captions_path, 'r') as file:
    # Lewati baris pertama (header) dan baca baris selanjutnya satu per satu
    for line in file.readlines()[1:]:
        # Pisahkan baris menjadi nama gambar dan caption berdasarkan koma pertama
        img_name, caption = line.strip().split(",", 1)
        # Tambahkan token "startseq" di awal dan "endseq" di akhir setiap caption
        caption = "startseq " + caption.strip() + " endseq"
        
        # Jika nama gambar belum ada di dictionary, inisialisasi daftar untuk caption
        if img_name not in captions_dict:
            captions_dict[img_name] = []
        # Tambahkan caption ke daftar caption untuk gambar tersebut
        captions_dict[img_name].append(caption)

In [4]:
# Initialize tokenizer and feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

d:\School Things\Uni\IlmuData2\deep-learning\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


# 3. Define Model (Vision Transformer + Text Decoder)

In [ ]:
# Inisialisasi Vision Encoder Decoder Model
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    "google/vit-base-patch16-224-in21k",  # Model Vision Transformer (ViT) sebagai encoder
    "bert-base-uncased"                  # Model BERT sebagai decoder
)

# Konfigurasi parameter model
# ID token awal untuk decoder, diambil dari tokenizer
model.config.decoder_start_token_id = tokenizer.cls_token_id
# ID token padding untuk mengisi token kosong
model.config.pad_token_id = tokenizer.pad_token_id
# Ukuran vocab decoder, diambil dari konfigurasi decoder
model.config.vocab_size = model.config.decoder.vocab_size
# ID token akhir, diambil dari tokenizer
model.config.eos_token_id = tokenizer.sep_token_id
# Panjang maksimal caption yang akan dihasilkan
model.config.max_length = 128

# Set parameter pelatihan tambahan
# Token awal bos (Beginning of Sequence), digunakan untuk memulai decoding
model.config.bos_token_id = tokenizer.cls_token_id
# Memaksa token awal bos untuk setiap decoding
model.config.forced_bos_token_id = tokenizer.cls_token_id

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

In [ ]:
# Inisialisasi optimizer AdamW
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
"""
Penjelasan:
- Optimizer AdamW digunakan untuk mengatur pembaruan parameter model selama pelatihan.
- AdamW adalah varian dari algoritma Adam dengan regularisasi weight decay untuk mengurangi overfitting.
- `model.parameters()`: Parameter model yang akan diperbarui.
- `lr=5e-5`: Learning rate, menentukan seberapa besar langkah pembaruan parameter.
"""

# Menentukan perangkat untuk pelatihan (GPU atau CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
"""
Penjelasan:
- `torch.device`: Menentukan perangkat yang akan digunakan untuk pelatihan.
- `torch.cuda.is_available()`: Mengecek apakah GPU tersedia di sistem.
- Jika GPU tersedia, pelatihan akan dilakukan di GPU. Jika tidak, akan dilakukan di CPU.
"""

# Memindahkan model ke perangkat yang ditentukan
model.to(device)
"""
Penjelasan:
- Memindahkan seluruh parameter model ke perangkat (GPU atau CPU) agar dapat diproses sesuai dengan perangkat yang tersedia.
- Jika menggunakan GPU, ini memungkinkan model memanfaatkan akselerasi hardware untuk pelatihan.
"""

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(i

**Data Loading**

> `batch_size` of **4** will use ~5gb of memory

> `batch_size` of **8** will use ~5.6gb of memory

> `batch_size` of **16** will use ~8gb of memory

In [ ]:
# Membagi data menjadi train dan test set
train_captions, test_captions = train_test_split(
    list(captions_dict.items()),  # Mengubah captions_dict menjadi list pasangan (img_name, captions)
    test_size=0.2,               # Menggunakan 20% data untuk test set
    random_state=69              # Seed random untuk memastikan hasil pembagian yang konsisten
)
"""
Penjelasan:
- `train_test_split`: Fungsi untuk membagi data menjadi train set dan test set.
- `list(captions_dict.items())`: Mengubah dictionary captions_dict menjadi list pasangan (image_name, captions).
- `test_size=0.2`: 20% data digunakan sebagai test set, sisanya untuk train set.
- `random_state=69`: Menentukan seed untuk membagi data secara acak dengan hasil yang konsisten.
"""

# Membuat objek dataset untuk training dan testing
train_dataset = Flickr8kDataset(dict(train_captions), images_path, feature_extractor, tokenizer)
test_dataset = Flickr8kDataset(dict(test_captions), images_path, feature_extractor, tokenizer)
"""
Penjelasan:
- `Flickr8kDataset`: Kelas dataset yang memproses gambar dan caption.
- `dict(train_captions)`: Mengubah list pasangan (img_name, captions) kembali menjadi dictionary untuk training.
- `dict(test_captions)`: Mengubah list pasangan (img_name, captions) kembali menjadi dictionary untuk testing.
- Parameter lain:
  - `images_path`: Path ke direktori gambar.
  - `feature_extractor`: Model pretrained untuk memproses gambar.
  - `tokenizer`: Tokenizer untuk memproses teks caption.
"""

# Membuat DataLoader untuk training dan testing
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)
"""
Penjelasan:
- `DataLoader`: Membungkus dataset untuk mempermudah iterasi selama pelatihan.
- Parameter:
  - `train_dataset` dan `test_dataset`: Dataset untuk training dan testing.
  - `batch_size=8`: Jumlah data yang diproses sekaligus dalam satu batch.
  - `shuffle=True` (hanya untuk training): Mengacak data untuk memastikan pelatihan lebih bervariasi.
"""

In [ ]:
# Jumlah epoch untuk pelatihan
epochs = 5
"""
Penjelasan:
- `epochs`: Jumlah iterasi penuh melalui seluruh dataset selama pelatihan.
- Dalam hal ini, model akan dilatih selama 5 epoch.
"""

# Loop utama untuk setiap epoch
for epoch in range(epochs):
    # Mengatur model dalam mode pelatihan
    model.train()
    # Inisialisasi total loss untuk epoch ini
    total_loss = 0
    # Membuat iterator dengan progres bar menggunakan tqdm
    batch_iterator = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    # Loop untuk setiap batch dalam DataLoader
    for pixel_values, input_ids, attention_mask in batch_iterator:
        # Memindahkan data ke perangkat (CPU/GPU)
        pixel_values = pixel_values.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Forward pass: Menghitung output dan loss
        outputs = model(
            pixel_values=pixel_values,        # Tensor gambar yang sudah diproses
            labels=input_ids,                 # Caption sebagai label
            decoder_attention_mask=attention_mask  # Attention mask untuk decoder
        )
        # Loss dari output model
        loss = outputs.loss
        # Tambahkan loss batch ke total_loss
        total_loss += loss.item()

        # Backward pass: Menghitung gradien
        loss.backward()
        # Update parameter model dengan optimizer
        optimizer.step()
        # Reset gradien untuk batch berikutnya
        optimizer.zero_grad()

        # Menampilkan nilai loss untuk batch ini di tqdm
        batch_iterator.set_postfix(batch_loss=loss.item())

    # Menampilkan loss rata-rata untuk epoch ini
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/5, Loss: 0.29994874144587735


Epoch 2/5, Loss: 0.27464860090145515


Epoch 3/5, Loss: 0.25878962841257913


Epoch 4/5, Loss: 0.24435266159815605


Epoch 5/5, Loss: 0.23278936003283016


In [ ]:
# Menyimpan model yang sudah dilatih, tokenizer, dan feature extractor
model.save_pretrained("./img_caption_googlevit_bert_tts")
"""
Penjelasan:
- `save_pretrained`: Fungsi untuk menyimpan model beserta konfigurasinya.
- `./img_caption_googlevit_bert_tts`: Direktori tujuan untuk menyimpan model.
- Hasil penyimpanan meliputi:
  - File konfigurasi model (`config.json`).
  - Parameter model dalam format PyTorch (`pytorch_model.bin`).
"""

tokenizer.save_pretrained("./img_caption_googlevit_bert_tts")
"""
Penjelasan:
- Menyimpan tokenizer yang digunakan untuk memproses caption.
- Hasil penyimpanan meliputi:
  - File konfigurasi tokenizer (`tokenizer_config.json`).
  - File vocab/tokenizer (`vocab.txt` atau file terkait lainnya).
- Menjamin bahwa tokenizer yang sama dapat digunakan saat model diload kembali.
"""

feature_extractor.save_pretrained("./img_caption_googlevit_bert_tts")
"""
Penjelasan:
- Menyimpan feature extractor yang digunakan untuk memproses gambar.
- Hasil penyimpanan meliputi:
  - File konfigurasi extractor (`preprocessor_config.json` atau serupa).
- Memastikan proses preprocessing gambar dapat direproduksi dengan presisi yang sama saat model digunakan kembali.
"""

d:\School Things\Uni\IlmuData2\deep-learning\Lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'forced_bos_token_id': 101}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


['./img_caption_googlevit_bert_tts\\preprocessor_config.json']

In [10]:
# Memuat kembali model Vision Encoder Decoder
model = VisionEncoderDecoderModel.from_pretrained("./img_caption_tf_googlevit_bert")
"""
Penjelasan:
- `from_pretrained`: Fungsi untuk memuat model yang telah disimpan sebelumnya.
- "./img_caption_tf_googlevit_bert": Direktori tempat model yang telah dilatih disimpan.
- Hasil pemuatan meliputi konfigurasi dan parameter model.
"""

# Memuat tokenizer dari direktori yang sama
tokenizer = AutoTokenizer.from_pretrained("./img_caption_tf_googlevit_bert")
"""
Penjelasan:
- `from_pretrained`: Memuat tokenizer yang digunakan saat pelatihan.
- Memastikan proses tokenisasi teks sama dengan yang digunakan selama pelatihan.
"""

# Memuat feature extractor dari model pretrained
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
"""
Penjelasan:
- `from_pretrained`: Memuat feature extractor Vision Transformer.
- Feature extractor memastikan gambar diproses ke format yang sesuai untuk encoder.
"""

# Mengonfigurasi token awal, akhir, padding, dan lainnya untuk decoder
model.config.decoder_start_token_id = tokenizer.cls_token_id
"""
Penjelasan:
- `decoder_start_token_id`: Token awal yang digunakan oleh decoder untuk memulai prediksi teks.
"""

model.config.bos_token_id = tokenizer.cls_token_id
"""
Penjelasan:
- `bos_token_id`: Token BOS (Beginning of Sequence) menandai awal teks untuk decoding.
"""

model.config.pad_token_id = tokenizer.pad_token_id
"""
Penjelasan:
- `pad_token_id`: Token untuk padding digunakan saat teks lebih pendek dari panjang maksimum.
"""

model.config.eos_token_id = tokenizer.sep_token_id
"""
Penjelasan:
- `eos_token_id`: Token EOS (End of Sequence) menandai akhir teks untuk decoding.
"""

model.generation_config.decoder_start_token_id = tokenizer.cls_token_id
"""
Penjelasan:
- `generation_config`: Konfigurasi khusus untuk proses teks generation.
- `decoder_start_token_id`: Token awal untuk setiap caption yang dihasilkan.
"""

# Menentukan perangkat (GPU atau CPU) untuk pelatihan atau inferensi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
"""
Penjelasan:
- `torch.device`: Menentukan perangkat yang akan digunakan untuk pemrosesan.
- Jika GPU tersedia, `cuda` akan digunakan. Jika tidak, CPU akan digunakan.
"""

# Memindahkan model ke perangkat yang ditentukan
model.to(device)
"""
Penjelasan:
- Memindahkan seluruh parameter model ke perangkat yang sesuai (GPU/CPU).
- Memastikan kompatibilitas antara data dan perangkat pemrosesan.
"""

d:\School Things\Uni\IlmuData2\deep-learning\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


'\nPenjelasan:\n- Memindahkan seluruh parameter model ke perangkat yang sesuai (GPU/CPU).\n- Memastikan kompatibilitas antara data dan perangkat pemrosesan.\n'

In [11]:
# Konfigurasi model untuk decoding
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.bos_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
"""
Penjelasan:
- `decoder_start_token_id`: Token awal untuk memulai decoding.
- `bos_token_id`: Token BOS (Beginning of Sequence) yang menandai awal teks.
- `eos_token_id`: Token EOS (End of Sequence) yang menandai akhir teks.
- `pad_token_id`: Token padding untuk mengisi token kosong dalam teks pendek.
"""

# Konfigurasi tambahan untuk proses generasi
model.generation_config.decoder_start_token_id = tokenizer.cls_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id
model.generation_config.eos_token_id = tokenizer.sep_token_id
"""
Penjelasan:
- `generation_config`: Digunakan untuk mengatur parameter generasi teks.
- `decoder_start_token_id`: Token awal untuk setiap caption yang dihasilkan.
- `pad_token_id`: Token padding untuk memastikan teks seragam.
- `eos_token_id`: Token akhir untuk menandai selesainya caption.
"""

# Fungsi untuk mengevaluasi model
def evaluate_model(model, dataloader, tokenizer, feature_extractor, device):
    model.eval()  # Mengatur model ke mode evaluasi
    actual, predicted = [], []  # List untuk menyimpan caption asli dan hasil prediksi

    with torch.no_grad():  # Tidak menghitung gradien selama evaluasi
        for pixel_values, input_ids, attention_mask in tqdm(dataloader, desc="Evaluating", leave=False):
            # Pindahkan data ke perangkat (CPU/GPU)
            pixel_values = pixel_values.to(device)
            captions = input_ids.tolist()  # Konversi tensor ke list ID token

            # Generate captions
            output_ids = model.generate(pixel_values, max_length=128, num_beams=5)
            """
            Penjelasan:
            - `generate`: Fungsi untuk menghasilkan teks dari model.
            - `max_length=128`: Panjang maksimal caption yang dihasilkan.
            - `num_beams=5`: Menggunakan beam search untuk menghasilkan teks terbaik.
            """

            # Decode hasil prediksi menjadi teks
            decoded_predictions = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]

            # Decode caption asli menjadi teks
            decoded_actual = [tokenizer.decode(ids, skip_special_tokens=True) for ids in captions]

            # Tambahkan hasil ke list actual dan predicted
            predicted.extend(decoded_predictions)
            actual.extend([[caption] for caption in decoded_actual])  # BLEU membutuhkan list of list

    # Menghitung skor BLEU
    bleu_score = corpus_bleu(actual, predicted)
    """
    Penjelasan:
    - `corpus_bleu`: Fungsi dari nltk untuk menghitung skor BLEU.
    - `actual`: Caption asli (list of list).
    - `predicted`: Caption yang dihasilkan oleh model.
    """
    print(f"BLEU Score: {bleu_score:.4f}")
    return bleu_score  # Mengembalikan skor BLEU

# Fungsi untuk menghasilkan caption dari gambar
def generate_caption(image_path, model, tokenizer, feature_extractor, max_length=128):
    # Preprocessing gambar
    image = Image.open(image_path).convert("RGB")  # Membuka gambar dan mengonversi ke RGB
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values.to(device)
    """
    Penjelasan:
    - Gambar diproses menggunakan feature_extractor agar sesuai dengan input encoder.
    - `return_tensors="pt"`: Mengembalikan tensor PyTorch.
    """

    # Generate caption dari gambar
    output_ids = model.generate(pixel_values, max_length=max_length, num_beams=5)
    """
    Penjelasan:
    - `generate`: Menghasilkan teks berdasarkan gambar yang diproses.
    - `max_length`: Panjang maksimal teks yang dihasilkan.
    - `num_beams`: Beam search untuk menghasilkan teks terbaik.
    """

    # Decode hasil prediksi menjadi teks
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Membersihkan teks dengan menghapus token tambahan
    caption = caption.replace("startseq", "").replace("endseq", "").strip()
    return caption  # Mengembalikan teks caption

In [25]:
bleu = evaluate_model(model, test_loader, tokenizer, feature_extractor, device)

Evaluating:   0%|          | 0/203 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


BLEU Score: 0.4614


In [ ]:
# Inisialisasi list untuk menyimpan sampel
samples = []
"""
Penjelasan:
- `samples`: List untuk menyimpan data evaluasi, termasuk nama gambar, caption asli, dan caption hasil prediksi.
"""

# Iterasi melalui beberapa data uji
for i in range(5):  # Ubah range untuk menyimpan lebih banyak contoh
    img_name, captions = test_captions[i]  # Ambil nama gambar dan caption dari test set
    img_path = os.path.join(images_path, img_name)  # Buat path lengkap gambar

    # Hasilkan caption untuk gambar
    generated_caption = generate_caption(img_path, model, tokenizer, feature_extractor)

    # Simpan data gambar, caption asli, dan caption hasil prediksi ke dalam list
    samples.append({
        "image": img_name,          # Nama file gambar
        "ground_truth": captions,   # Caption asli (daftar caption)
        "generated": generated_caption  # Caption yang dihasilkan model
    })
"""
Penjelasan:
- Iterasi ini mengambil beberapa gambar dari test set dan menghasilkan caption untuk masing-masing.
- Caption yang dihasilkan dibandingkan dengan caption asli.
"""

# Simpan sampel evaluasi ke file JSON
with open("evaluation_samples.json", "w") as f:
    json.dump(samples, f, indent=4)
"""
Penjelasan:
- `json.dump`: Menyimpan data `samples` ke dalam file JSON.
- `indent=4`: Mengatur format file JSON agar lebih mudah dibaca.
- "evaluation_samples.json": Nama file tempat data evaluasi disimpan.
"""

In [21]:
# Test the function
test_image_path = "example_pics/ello_gitar.jpg"  # Replace with your image path
print("Generated Caption:", generate_caption(test_image_path, model, tokenizer, feature_extractor))

Generated Caption: a man playing guitar.
